# Revisión de datos de los archivos csv de Open Contracting Partnership. Data Set del 2018.

Usando el buscador de OpenContracting se buscaran datos sobre Chile (buscador: https://data.open-contracting.org/es/search/). Se decidió descargarlos archivos de los años 2018 y 2021, para tener una muestra de datos que contenga información sobre licitaciones en un rango de tiempo cercano a estas fechas.

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
%cd Chile
%cd 2018
%ls
%cd ..
%cd ..

c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile
c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile\2018
 El volumen de la unidad C es OS
 El n�mero de serie del volumen es: 240D-CD63

 Directorio de c:\Users\camil\OneDrive\Documentos\GitHub\Proyecto-IMT-2200-1-Grupo-14\Datos y trabajo con muestras\Chile\2018

18-11-2024  15:43    <DIR>          .
18-11-2024  15:05    <DIR>          ..
18-11-2024  15:43        64.571.337 awards.csv
18-11-2024  15:29       137.878.898 awards_documents.csv
18-11-2024  15:29       172.974.592 awards_items.csv
18-11-2024  15:26        30.901.082 awards_suppliers.csv
18-11-2024  15:29       165.819.300 main.csv
18-11-2024  15:31       370.417.597 parties.csv
18-11-2024  15:30       243.504.970 parties_additionalIdentifiers.csv
18-11-2024  15:30       287.507.728 tender_items.csv
18-11-2024  15:27       103.927.462 tender_tenderers.csv
            

In [3]:
from os import path
import pandas as pd

lista_doc = ["main.csv","awards.csv", "awards_items.csv", "awards_suppliers.csv"]
contador = 0
for documento in lista_doc:
    print(documento)
    camino = path.join("Chile","2018",documento)
    if contador == 0:
        df_main = pd.read_csv(camino, delimiter=",")
        
    elif contador == 1:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas = list(df_e.columns.values)
        
        if "description" in lista_columnas :
            df_e.rename(columns={'description': f'description_{documento}'}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
        df_main = df_main.merge(df_e, left_on="_link", right_on="_link_main")
        
    else:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas =  list(df_e.columns.values)
        if "description" in lista_columnas:
            df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}'}, inplace=True)
        df_main = df_main.merge(df_e, left_on="_link_awards.csv", right_on="_link_awards")
    contador += 1
    
df_main.info()

main.csv
awards.csv
awards_items.csv
awards_suppliers.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2682185 entries, 0 to 2682184
Data columns (total 58 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   _link_x                              object 
 1   id                                   object 
 2   tag                                  object 
 3   date_x                               object 
 4   ocid                                 object 
 5   language                             object 
 6   initiationType                       object 
 7   buyer_id                             object 
 8   buyer_name                           object 
 9   tender_id                            object 
 10  tender_procurementMethodDetails      object 
 11  tender_title                         object 
 12  tender_status                        object 
 13  tender_procurementMethod             object 
 14  tender_hasEnquiries     

Se cargan los documentos main, awards, award_items y award_suppliers, a un DataFrame, se limpiaran los datos haciendo drop de columnas irrelevantes y/o repetidas.

In [4]:
df_main.drop(columns=["id","ocid","tender_procuringEntity_id", "_link_awards_items.csv","id_awards_items.csv", "tender_id", "buyer_id", "_link_awards_x", "_link_awards_y"], inplace=True)
df_main.drop(columns=["classification_uri", "_link_awards.csv", "id_awards.csv", "id_awards_suppliers.csv", "unit_value_unitOfAccount", "value_unitOfAccount", "language" ], inplace=True)
df_main.columns

Index(['_link_x', 'tag', 'date_x', 'initiationType', 'buyer_name',
       'tender_procurementMethodDetails', 'tender_title', 'tender_status',
       'tender_procurementMethod', 'tender_hasEnquiries', 'tender_description',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_procuringEntity_name',
       'tender_value_amount', 'tender_value_currency',
       'tender_value_unitOfAccount', '_link_main_x', 'date_y',
       'description_awards.csv', 'title', 'status', 'value_amount',
       'value_currency', '_link_main_y', 'description', 'quantity',
       'unit_name', 'unit_value_amount', 'unit_value_currency',
       'classification_id', 'classification_scheme', '_link_y', '_link_main',
       'na

Se eliminan las columnas con datos que permiten la identificación dentro del sistema del que se originan, puesto que no son pertinentes para los analisis a realizarse.

In [5]:
df_main[["_link_main", "_link_y", "_link_main_y", "_link_main_x", "_link_x"]]

,_link_main,_link_y,_link_main_y,_link_main_x,_link_x
0,id-0.0,id-0.0.awards.0.suppliers.0,id-0.0,id-0.0,id-0.0
1,id-0.2,id-0.2.awards.0.suppliers.0,id-0.2,id-0.2,id-0.2
2,id-0.4,id-0.4.awards.0.suppliers.0,id-0.4,id-0.4,id-0.4
3,id-0.5,id-0.5.awards.0.suppliers.0,id-0.5,id-0.5,id-0.5
4,id-0.5,id-0.5.awards.0.suppliers.1,id-0.5,id-0.5,id-0.5
...,...,...,...,...,...
2682180,id-15.15080,id-15.15080.awards.0.suppliers.1,id-15.15080,id-15.15080,id-15.15080
2682181,id-15.15080,id-15.15080.awards.0.suppliers.2,id-15.15080,id-15.15080,id-15.15080
2682182,id-15.15080,id-15.15080.awards.0.suppliers.3,id-15.15080,id-15.15080,id-15.15080
2682183,id-15.15080,id-15.15080.awards.0.suppliers.4,id-15.15080,id-15.15080,id-15.15080


Se revisa el contenido de las columnas "_link_main", "_link_y", "_link_main_y", "_link_main_x", "_link_x y se observa que _link_main y las columnas derivadas de este nombre son identicas entre ellas y con la columna _link_x, se eliminaran todas excepto main, ademas la columna _link_y no es relevancia para el DataFrame, puesto que es identica a la columna _link_main.

In [6]:
df_main.drop(columns=["_link_y", "_link_main_y", "_link_main_x", "_link_x"], inplace=True)
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2682185 entries, 0 to 2682184
Data columns (total 38 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   tag                                  object 
 1   date_x                               object 
 2   initiationType                       object 
 3   buyer_name                           object 
 4   tender_procurementMethodDetails      object 
 5   tender_title                         object 
 6   tender_status                        object 
 7   tender_procurementMethod             object 
 8   tender_hasEnquiries                  bool   
 9   tender_description                   object 
 10  tender_awardPeriod_endDate           object 
 11  tender_awardPeriod_startDate         object 
 12  tender_awardPeriod_durationInDays    float64
 13  tender_tenderPeriod_endDate          object 
 14  tender_tenderPeriod_startDate        object 
 15  tender_tenderPeriod_durationInDa

In [7]:
df_main[["date_y", "date_x"]]

,date_y,date_x
0,2017-12-23T13:23:24Z,2018-12-31T11:27:18Z
1,2018-02-02T11:16:54Z,2018-12-31T11:29:05Z
2,2017-12-21T18:17:23Z,2018-12-31T11:20:54Z
3,2018-01-16T10:56:54Z,2018-12-31T11:29:33Z
4,2018-01-16T10:56:54Z,2018-12-31T11:29:33Z
...,...,...
2682180,2016-02-19T16:06:59Z,2018-12-31T11:49:56Z
2682181,2016-02-19T16:06:59Z,2018-12-31T11:49:56Z
2682182,2016-02-19T16:06:59Z,2018-12-31T11:49:56Z
2682183,2016-02-19T16:06:59Z,2018-12-31T11:49:56Z


Se comparan _date_y y _date_x y se observa que son distintas por lo que se mantienen, pero se cambia el nombre.

In [8]:
df_main.rename(columns={'date_y': 'date_award', "date_x": 'date_main'}, inplace=True)
df_main.drop(columns=["tag"], inplace=True)
df_main.rename(columns={"name": " Potential Provider name"}, inplace= True)

In [9]:
df_main["buyer == procuring_entity"] = df_main["buyer_name"] == df_main["tender_procuringEntity_name"]
df_main["buyer == procuring_entity"].unique()

array([ True])

Se revisa si las columnas "tender_procuringEntity_name" y "buyer_name" son iguales, como lo son se eliminará "tender_procuringEntity_name".

In [10]:
df_main.drop(columns=["buyer == procuring_entity", "tender_procuringEntity_name"], inplace=True)

In [11]:
df_clasificaciones = pd.read_excel("unspcs-clasificador-de-bienes-y-servicios-de-naciones-unidas-en-espanol.xlsx")
df_clasificaciones.head(10)

,NaN,Unnamed: 1,"Una combinación de códigos UNSPSC y palabras clave, es la mejor solución para una eficiente búsqueda de licitaciones. Prueba nuestro servicio de alerta por mail y no pierdas de vista ninguna convocatoria.",Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,*El Programa de las Naciones Unidas para el De...,NaN,NaN,NaN,NaN,NaN
4,Código segmento,Nombre Segmento,Código Familia,Nombre Familia,Código Clase,Nombre Clase,Código Producto,Nombre Producto
5,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101501,Gatos
6,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101502,Perros
7,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101504,Visón
8,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101505,Ratas
9,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101506,Caballos


Se observa que las columnas del dataframe estan mal nombradas y que los nombres de las columnas se encuentran en la 4° fila del dataframe.

In [12]:
df_clasificaciones.drop([0,1,2,3],inplace=True)
df_clasificaciones = df_clasificaciones.set_axis(["Código segmento","Nombre Segmento","Código Familia","Nombre Familia", "Código Clase", "Nombre Clase",
                                 "Código Producto", "Nombre Producto"], axis=1)
df_clasificaciones.drop(4,inplace=True)
df_clasificaciones

,Código segmento,Nombre Segmento,Código Familia,Nombre Familia,Código Clase,Nombre Clase,Código Producto,Nombre Producto
5,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101501,Gatos
6,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101502,Perros
7,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101504,Visón
8,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101505,Ratas
9,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101506,Caballos
...,...,...,...,...,...,...,...,...
49022,95,"Terrenos, Edificios, Estructuras y Vías",9514,Estructuras y edificios prefabricados,951418,Edificios y estructuras prefabricados de emerg...,95141803,Unidad de contenedor
49023,95,"Terrenos, Edificios, Estructuras y Vías",9514,Estructuras y edificios prefabricados,951419,Edificios y estructuras prefabricados médicos,95141901,Unidad médica
49024,95,"Terrenos, Edificios, Estructuras y Vías",9514,Estructuras y edificios prefabricados,951419,Edificios y estructuras prefabricados médicos,95141902,Unidad de laboratorio
49025,95,"Terrenos, Edificios, Estructuras y Vías",9514,Estructuras y edificios prefabricados,951419,Edificios y estructuras prefabricados médicos,95141903,Unidad odontológica


El dataframe df_clasificaciones corresponde a un conjunto de datos que posee el significado de cada objeto solicitados según la codificación UNSPSC, por lo que se hara merge con el df_main para saber que se esta comprando en cada licitación, se desecharán algunas de las columnas de df_clasificaciones para esto.

In [13]:
df_clasificaciones.drop(columns=["Código segmento","Código Familia","Nombre Familia","Nombre Clase","Código Clase"], inplace=True)
# se mantiene solo el codigo del producto en particular, puesto que este es de 8 digitos al igual que el classification_id del df_main

In [14]:
df_main = df_main.merge(df_clasificaciones, left_on="classification_id", right_on="Código Producto")

Ya habiendo realizado el merge, se procede a quitar las columnas que ya se utilizaron para realizar este, (classification_id, classification_scheme, Código Producto) y las columnas tender_title, description y tender_description puesto que no se necesita una descripción adicional de la de Nombre Clase de los items que se buscan adquirir

In [15]:
df_main.drop(columns=["classification_id", "classification_scheme", "Código Producto", "tender_title", "description", "tender_description"], inplace=True)
df_main.head()

,date_main,initiationType,buyer_name,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,tender_awardPeriod_durationInDays,...,value_amount,value_currency,quantity,unit_name,unit_value_amount,unit_value_currency,_link_main,Potential Provider name,Nombre Segmento,Nombre Producto
0,2018-12-31T11:27:18Z,tender,Ilustre Municipalidad de Quintero | UNIDAD NORMAL,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2017-12-15T18:05:00Z,2017-12-13T09:03:00Z,1.0,...,690000.0,CLP,1.0,Unidad,690000.0,CLP,id-0.0,JOSÉ MIGUEL | JOSÉ MIGUEL,"Instrumentos Musicales, Juegos, Juguetes, Arte...",Juguetes inflables
1,2018-12-31T11:29:05Z,tender,ILUSTRE MUNICIPALIDAD DE LA GRANJA | Adquisici...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2018-02-02T19:38:00Z,2018-01-02T17:00:00Z,1.0,...,870000.0,CLP,1.0,Unidad,870000.0,CLP,id-0.2,Todovidrios | Vidrieria las condes 7002,"Componentes y Suministros para Estructuras, Ed...",Puertas de cristal
2,2018-12-31T11:20:54Z,tender,Municipalidad de Los Angeles | MUNICIPALIDAD D...,Licitación Pública Entre 100 y 1000 UTM (LE),complete,open,True,2017-12-14T16:55:00Z,2017-12-11T16:30:00Z,1.0,...,6000000.0,CLP,1.0,Unidad,6000000.0,CLP,id-0.4,Jorge Alejandro | Jorge Alejandro,Servicios Educativos y de Formación,Servicios de asesorías educativas
3,2018-12-31T11:29:33Z,tender,Ilustre Municipalidad de Cochamo | Ilustre Mun...,Licitación Pública Menor a 100 UTM (L1),complete,open,True,2017-12-29T16:00:00Z,2017-12-28T12:02:00Z,1.0,...,347043.0,CLP,4.0,Unidad,4995.0,CLP,id-0.5,INSTITUTO ORTOPEDICO Y DE REHABILITACION IOR L...,"Componentes y Suministros para Estructuras, Ed...",Placa de aleación ferrosa
4,2018-12-31T11:29:33Z,tender,Ilustre Municipalidad de Cochamo | Ilustre Mun...,Licitación Pública Menor a 100 UTM (L1),complete,open,True,2017-12-29T16:00:00Z,2017-12-28T12:02:00Z,1.0,...,347043.0,CLP,4.0,Unidad,4995.0,CLP,id-0.5,RICARDO RODRIGUEZ Y CIA. LTDA. | RICARDO RODRI...,"Componentes y Suministros para Estructuras, Ed...",Placa de aleación ferrosa


In [16]:
duplicates = df_main.duplicated(keep=False)
dups = df_main[duplicates]
dups.head(10)

,date_main,initiationType,buyer_name,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,tender_awardPeriod_durationInDays,...,value_amount,value_currency,quantity,unit_name,unit_value_amount,unit_value_currency,_link_main,Potential Provider name,Nombre Segmento,Nombre Producto
235,2018-12-31T11:25:48Z,tender,Ilustre Municipalidad de Punitaqui | Ilustre M...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2017-12-12T20:01:00Z,2017-12-11T20:01:00Z,1.0,...,59900.0,CLP,200.0,Unidad,15.0,CLP,id-0.52,imprenta zaimor y cía. ltda | imprenta zaimor ...,"Servicios Editoriales, de Diseño, de Artes Gra...",Impresión de papelería o formularios comerciales
236,2018-12-31T11:25:48Z,tender,Ilustre Municipalidad de Punitaqui | Ilustre M...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2017-12-12T20:01:00Z,2017-12-11T20:01:00Z,1.0,...,59900.0,CLP,200.0,Unidad,15.0,CLP,id-0.52,imprenta zaimor y cía. ltda | imprenta zaimor ...,"Servicios Editoriales, de Diseño, de Artes Gra...",Impresión de papelería o formularios comerciales
237,2018-12-31T11:25:48Z,tender,Ilustre Municipalidad de Punitaqui | Ilustre M...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2017-12-12T20:01:00Z,2017-12-11T20:01:00Z,1.0,...,59900.0,CLP,200.0,Unidad,15.0,CLP,id-0.52,imprenta zaimor y cía. ltda | imprenta zaimor ...,"Servicios Editoriales, de Diseño, de Artes Gra...",Impresión de papelería o formularios comerciales
328,2018-12-31T11:24:04Z,tender,I. MUNICIPALIDAD DE TALCA | ADQUISICIONES MUNI...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2018-01-12T15:48:00Z,2017-12-19T13:51:00Z,1.0,...,1664234.0,CLP,100.0,kilogramo,1075.0,CLP,id-0.84,Dipagro SpA | Dipagro SpA,Medicamentos y Productos Farmacéuticos,Butilbromuro de hioscina
329,2018-12-31T11:24:04Z,tender,I. MUNICIPALIDAD DE TALCA | ADQUISICIONES MUNI...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2018-01-12T15:48:00Z,2017-12-19T13:51:00Z,1.0,...,1664234.0,CLP,100.0,kilogramo,1075.0,CLP,id-0.84,Dipagro SpA | Dipagro SpA,Medicamentos y Productos Farmacéuticos,Butilbromuro de hioscina
330,2018-12-31T11:24:04Z,tender,I. MUNICIPALIDAD DE TALCA | ADQUISICIONES MUNI...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2018-01-12T15:48:00Z,2017-12-19T13:51:00Z,1.0,...,1664234.0,CLP,100.0,kilogramo,1075.0,CLP,id-0.84,Dipagro SpA | Dipagro SpA,Medicamentos y Productos Farmacéuticos,Butilbromuro de hioscina
410,2018-12-31T11:28:39Z,tender,Ilustre Municipalidad de Los Sauces | Ilustre ...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2017-12-22T17:36:00Z,2017-12-21T17:35:00Z,1.0,...,62500.0,CLP,6.0,Unidad,2150.0,CLP,id-0.156,Allmedica S.A | Allmedica S.A,"Maquinaria, Equipo y Suministros para la Indus...",Bandejas para servicio de comidas
411,2018-12-31T11:28:39Z,tender,Ilustre Municipalidad de Los Sauces | Ilustre ...,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2017-12-22T17:36:00Z,2017-12-21T17:35:00Z,1.0,...,62500.0,CLP,6.0,Unidad,2150.0,CLP,id-0.156,Allmedica S.A | Allmedica S.A,"Maquinaria, Equipo y Suministros para la Indus...",Bandejas para servicio de comidas
460,2018-12-31T11:26:29Z,tender,MUNICIPALIDAD DE DOÑIHUE | Salud,Licitación Pública Entre 100 y 1000 UTM (LE),complete,open,True,2018-01-15T19:58:00Z,2017-12-14T19:58:00Z,1.0,...,2600.0,CLP,100.0,Unidad,8.0,CLP,id-0.160,Marielle Patricia | Marielle Patricia,"Servicios Editoriales, de Diseño, de Artes Gra...",Impresión de papelería o formularios comerciales
462,2018-12-31T11:26:29Z,tender,MUNICIPALIDAD DE DOÑIHUE | Salud,Licitación Pública Entre 100 y 1000 UTM (LE),complete,open,True,2018-01-15T19:58:00Z,2017-12-14T19:58:00Z,1.0,...,2600.0,CLP,100.0,Unidad,8.0,CLP,id-0.160,Marielle Patricia | Marielle Patricia,"Servicios Editoriales, de Diseño, de Artes Gra...",Impresión de papelería o formularios comerciales


Se tiene en consideración que dentro de un mismo proceso de licitación pueden existir diferentes entidades que ofrecen cubrir las necesidades planteadas al publicar la licitación, por lo que se mantendran los duplicados.

### Modificación de tipos de datos

Se cambiarán las columnas que contienen información de fechas a objetos de tipo DateTime.

In [17]:
df_main.columns

Index(['date_main', 'initiationType', 'buyer_name',
       'tender_procurementMethodDetails', 'tender_status',
       'tender_procurementMethod', 'tender_hasEnquiries',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_value_amount',
       'tender_value_currency', 'tender_value_unitOfAccount', 'date_award',
       'description_awards.csv', 'title', 'status', 'value_amount',
       'value_currency', 'quantity', 'unit_name', 'unit_value_amount',
       'unit_value_currency', '_link_main', ' Potential Provider name',
       'Nombre Segmento', 'Nombre Producto'],
      dtype='object')

In [18]:
df_2018 = df_main.reset_index()
df_respaldo = df_main

In [19]:
df_2018["date_main"] = pd.to_datetime(df_2018['date_main'], format='%Y-%m-%dT%H:%M:%SZ')

In [20]:
df_2018["date_award"] = pd.to_datetime(df_2018['date_award'], format='%Y-%m-%dT%H:%M:%SZ')

In [21]:
df_2018["tender_enquiryPeriod_startDate"] = pd.to_datetime(df_2018['tender_enquiryPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

In [22]:
df_2018["tender_enquiryPeriod_endDate"] = pd.to_datetime(df_2018['tender_enquiryPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')

In [23]:
df_2018['tender_tenderPeriod_startDate'] = pd.to_datetime(df_2018['tender_tenderPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

Al parecer hay un error en la fila 1, eliminemosla.

In [24]:
df_2018 = df_2018.drop(df_2018.index[1])

In [25]:
df_2018["tender_tenderPeriod_endDate"] = pd.to_datetime(df_2018['tender_tenderPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')

In [26]:
df_2018['tender_awardPeriod_startDate'] = pd.to_datetime(df_2018['tender_awardPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

Hay un error en el formato de una fila con contenido <5017-05-05T17:00:00Z>, se eliminará.

In [27]:
df_2018[df_2018["tender_awardPeriod_endDate"] == "5017-05-05T17:00:00Z"].index


Index([1875296], dtype='int64')

In [28]:
df_2018.drop([1875296], axis=0, inplace=True)

In [29]:
df_2018['tender_awardPeriod_endDate'] = pd.to_datetime(df_2018['tender_awardPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')

In [30]:
df_2018['tender_enquiryPeriod_durationInDays'] = df_2018['tender_enquiryPeriod_endDate'] - df_2018['tender_enquiryPeriod_startDate'] 

In [31]:
df_2018['tender_awardPeriod_durationInDays'] = df_2018['tender_awardPeriod_endDate'] - df_2018['tender_awardPeriod_startDate']

In [32]:
df_2018['tender_tenderPeriod_durationInDays'] = df_2018['tender_enquiryPeriod_endDate'] - df_2018['tender_tenderPeriod_startDate']

## Filtro según fecha

Aqui realizaremos un filtro de las licitaciones en base a cuando se iniciaron y cuando terminaron respecto a lo relacionado al 2018. En este caso seleccionaremos las licitaciones que partan almenos desde mitad de 2017 hasta el final del 2018.

In [33]:
import datetime

In [34]:
fecha_filtro = '2017-06-01'
df_filtrado = df_2018.loc[df_2018['tender_tenderPeriod_startDate'] >= fecha_filtro]

In [35]:
fecha_filtro = '2019-01-01'
df_filtrado = df_filtrado.loc[df_filtrado['tender_tenderPeriod_endDate'] < fecha_filtro]

In [36]:
df_filtrado.head()

,index,date_main,initiationType,buyer_name,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,value_amount,value_currency,quantity,unit_name,unit_value_amount,unit_value_currency,_link_main,Potential Provider name,Nombre Segmento,Nombre Producto
0,0,2018-12-31 11:27:18,tender,Ilustre Municipalidad de Quintero | UNIDAD NORMAL,Licitación Pública Menor a 100 UTM (L1),complete,open,False,2017-12-15 18:05:00,2017-12-13 09:03:00,...,690000.0,CLP,1.0,Unidad,690000.0,CLP,id-0.0,JOSÉ MIGUEL | JOSÉ MIGUEL,"Instrumentos Musicales, Juegos, Juguetes, Arte...",Juguetes inflables
2,2,2018-12-31 11:20:54,tender,Municipalidad de Los Angeles | MUNICIPALIDAD D...,Licitación Pública Entre 100 y 1000 UTM (LE),complete,open,True,2017-12-14 16:55:00,2017-12-11 16:30:00,...,6000000.0,CLP,1.0,Unidad,6000000.0,CLP,id-0.4,Jorge Alejandro | Jorge Alejandro,Servicios Educativos y de Formación,Servicios de asesorías educativas
3,3,2018-12-31 11:29:33,tender,Ilustre Municipalidad de Cochamo | Ilustre Mun...,Licitación Pública Menor a 100 UTM (L1),complete,open,True,2017-12-29 16:00:00,2017-12-28 12:02:00,...,347043.0,CLP,4.0,Unidad,4995.0,CLP,id-0.5,INSTITUTO ORTOPEDICO Y DE REHABILITACION IOR L...,"Componentes y Suministros para Estructuras, Ed...",Placa de aleación ferrosa
4,4,2018-12-31 11:29:33,tender,Ilustre Municipalidad de Cochamo | Ilustre Mun...,Licitación Pública Menor a 100 UTM (L1),complete,open,True,2017-12-29 16:00:00,2017-12-28 12:02:00,...,347043.0,CLP,4.0,Unidad,4995.0,CLP,id-0.5,RICARDO RODRIGUEZ Y CIA. LTDA. | RICARDO RODRI...,"Componentes y Suministros para Estructuras, Ed...",Placa de aleación ferrosa
5,5,2018-12-31 11:29:33,tender,Ilustre Municipalidad de Cochamo | Ilustre Mun...,Licitación Pública Menor a 100 UTM (L1),complete,open,True,2017-12-29 16:00:00,2017-12-28 12:02:00,...,347043.0,CLP,4.0,Unidad,4995.0,CLP,id-0.5,María | María,"Componentes y Suministros para Estructuras, Ed...",Placa de aleación ferrosa


In [37]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 322543 entries, 0 to 2408231
Data columns (total 34 columns):
 #   Column                               Non-Null Count   Dtype          
---  ------                               --------------   -----          
 0   index                                322543 non-null  int64          
 1   date_main                            322543 non-null  datetime64[ns] 
 2   initiationType                       322543 non-null  object         
 3   buyer_name                           322543 non-null  object         
 4   tender_procurementMethodDetails      322543 non-null  object         
 5   tender_status                        322543 non-null  object         
 6   tender_procurementMethod             322543 non-null  object         
 7   tender_hasEnquiries                  322543 non-null  bool           
 8   tender_awardPeriod_endDate           322543 non-null  datetime64[ns] 
 9   tender_awardPeriod_startDate         322543 non-null  datetime6

## Limpieza y transformación de columnas

Se revisarán que las filas no posean solo un dato, si sucede, se eliminarán. (Se ignorarán las filas que evidentemente poseen más de un dato).

In [38]:
df_filtrado.columns

Index(['index', 'date_main', 'initiationType', 'buyer_name',
       'tender_procurementMethodDetails', 'tender_status',
       'tender_procurementMethod', 'tender_hasEnquiries',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_value_amount',
       'tender_value_currency', 'tender_value_unitOfAccount', 'date_award',
       'description_awards.csv', 'title', 'status', 'value_amount',
       'value_currency', 'quantity', 'unit_name', 'unit_value_amount',
       'unit_value_currency', '_link_main', ' Potential Provider name',
       'Nombre Segmento', 'Nombre Producto'],
      dtype='object')

In [39]:
df_de_uniques = {}

for columna in df_filtrado.columns:
    df_de_uniques[columna] = df_filtrado[columna].unique()

In [40]:
for columna in df_filtrado.columns:
    print(columna)
    print(df_de_uniques[columna])
    print("#####################################################################################################")

index
[      0       2       3 ... 2408229 2408230 2408231]
#####################################################################################################
date_main
<DatetimeArray>
['2018-12-31 11:27:18', '2018-12-31 11:20:54', '2018-12-31 11:29:33',
 '2018-12-31 11:28:39', '2018-12-31 11:27:20', '2018-12-31 11:23:53',
 '2018-12-31 11:29:19', '2018-12-31 11:29:15', '2018-12-31 11:25:11',
 '2018-12-31 11:25:39',
 ...
 '2018-12-31 10:30:35', '2018-12-31 10:39:03', '2018-12-31 10:36:48',
 '2018-12-31 10:30:21', '2018-12-31 10:39:56', '2018-12-31 10:36:25',
 '2018-12-31 10:21:59', '2018-12-31 10:38:35', '2018-12-31 10:37:42',
 '2018-12-31 10:23:20']
Length: 1291, dtype: datetime64[ns]
#####################################################################################################
initiationType
['tender']
#####################################################################################################
buyer_name
['Ilustre Municipalidad de Quintero | UNIDAD NORMAL'
 'Municip

In [41]:
df_2018_m = df_filtrado
df_filtrado_respaldo = df_filtrado

In [42]:
df_2018_m.loc[df_2018_m["tender_enquiryPeriod_durationInDays"].isna()]

,index,date_main,initiationType,buyer_name,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,value_amount,value_currency,quantity,unit_name,unit_value_amount,unit_value_currency,_link_main,Potential Provider name,Nombre Segmento,Nombre Producto


In [43]:
df_2018_m.loc[df_2018_m["tender_tenderPeriod_durationInDays"].isna()]

,index,date_main,initiationType,buyer_name,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,value_amount,value_currency,quantity,unit_name,unit_value_amount,unit_value_currency,_link_main,Potential Provider name,Nombre Segmento,Nombre Producto


In [44]:
df_2018_m.loc[df_2018_m["tender_awardPeriod_durationInDays"].isna()]

,index,date_main,initiationType,buyer_name,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,value_amount,value_currency,quantity,unit_name,unit_value_amount,unit_value_currency,_link_main,Potential Provider name,Nombre Segmento,Nombre Producto


In [45]:
df_2018_m["tender_awardPeriod_durationInDays"] = df_2018_m["tender_awardPeriod_durationInDays"].replace(np.nan, datetime.timedelta(days=0))

In [46]:
df_2018_m.drop(columns=["tender_status", "status", "initiationType", "tender_value_unitOfAccount", "description_awards.csv", "title"], inplace=True)

In [48]:
df_iguales = pd.DataFrame(df_2018_m["tender_value_amount"] == df_2018_m["value_amount"])
df_iguales.head()

,0
0,False
2,True
3,False
4,False
5,False


In [ ]:
df_iguales.rename(columns={0: "tender_value_amount == value_amount"})

## Limpieza de datos nulos

Se revisarán las filas con valores nulos.

In [ ]:
sin_nan = df_2018_m[[ 'tender_value_amount','tender_value_currency', 'value_amount', 'value_currency', 'quantity','unit_name', 'unit_value_amount', 'unit_value_currency']].dropna()
sin_nan.info()

In [ ]:
sin_nan.head()

In [ ]:
Datos_with_NaN = df_2018_m.loc[:, df_2018_m.isna().any()]
Datos_with_NaN.info()

In [ ]:
Datos_with_NaN.head()

In [ ]:
filas_con_nan = Datos_with_NaN[Datos_with_NaN.isna().any(axis=1)]

In [ ]:
print(f"La cantidad de datos NaN es de: {len(filas_con_nan)} en todo el dataframe de {len(df_2018_m)}")

Pues aqui nos fijamos que en todo el dataframe hay 235137 filas con datos NaN, por lo que será mejor no eliminarlos y evaluar que podemos hacer

In [ ]:
df_2018_m.columns

In [ ]:
df_2018_m.info()